In [6]:
import sys
from sys import platform
if platform  == 'win32':
    sys.argv=["_", r"P:\rmahajan14\capstone_data\data", r"P:\rmahajan14\capstone_data\cache"]
if platform == 'linux' or platform == 'linux2':
    def full_path(p):
        return os.path.abspath(os.path.expanduser(p))
    sys.argv=["_", r"/mnt/d/DiDi_Ride_Data/data", r"/mnt/d/DiDi_Ride_Data/Ride_Data_Cache"]

In [7]:
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from loader1 import *
from utils import *
from common import *
from experiment2 import *
# from sys import platform

import warnings
warnings.filterwarnings('ignore')

In [9]:
import time

Basic Coding 

In [10]:
spatial_type = 'radial'

In [14]:
df_final_no_decay, target_df_no_decay = get_final_df_reg(use_cache=True, decay='No Decay', spatial_type=spatial_type)

../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
No Decay
../Ride_Data_Cache/active_times.msgpack exists
Decay Calculation done in 1.0522358417510986
1e
../Ride_Data_Cache/features_orders.msgpack exists
Features created in 0.913564920425415
1f
../Ride_Data_Cache/radial_spatial_df.msgpack exists
spatial
Spatial Calculation done in 5.416323900222778


In [15]:
df_final_old_decay, target_df_old_decay = get_final_df_reg(use_cache=True, decay='Old Decay',spatial_type=spatial_type)

../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
Old Decay
../Ride_Data_Cache/idle_times_old.msgpack exists
Decay Calculation done in 0.9870419502258301
1e
../Ride_Data_Cache/features_orders.msgpack exists
Features created in 0.8762009143829346
1f
../Ride_Data_Cache/radial_spatial_df.msgpack exists
spatial
Spatial Calculation done in 3.07307505607605


In [16]:
# Using the function create_modified_active_time_through_decay2 
# instead of create_modified_active_time_through_decay
df_final_new_decay1, target_df_new_decay1 = get_final_df_reg(use_cache=True, decay='New Decay', mult_factor=1, spatial_type=spatial_type)
df_final_new_decay10, target_df_new_decay10 = get_final_df_reg(use_cache=True, decay='New Decay', mult_factor=10, spatial_type=spatial_type)
df_final_new_decay100, target_df_new_decay100 = get_final_df_reg(use_cache=True, decay='New Decay', mult_factor=100, spatial_type=spatial_type)

../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
New Decay
../Ride_Data_Cache/idle_times_new_1.msgpack exists
Decay Calculation done in 1.0030932426452637
1e
../Ride_Data_Cache/features_orders.msgpack exists
Features created in 0.8606500625610352
1f
../Ride_Data_Cache/radial_spatial_df.msgpack exists
spatial
Spatial Calculation done in 4.022770166397095
../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
New Decay
../Ride_Data_Cache/idle_times_new_10.msgpack exists
Decay Calculation done in 0.9495670795440674
1e
../Ride_Data_Cache/features_orders.msgpack exists
Features created in 0.8390188217163086
1f
../Ride_Data_Cache/radial_spatial_df.msgpack exists
spatial
Spatial Calculation done in 4.502440929412842
../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
New Decay
../Ride_Data_Cache/idle_times_new_100.msgpack exists
Decay Calculation done in 0.9557948112487793
1e
../Ride_Data_Cache/features_orders.msgpack exists
Featu

In [17]:
# Using the function create_modified_active_time_through_decay2 
# instead of create_modified_active_time_through_decay
df_final_new_decay_surv, target_df_new_decay_surv = get_final_df_reg(use_cache=True, decay='Survival', spatial_type=spatial_type)

../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
Survival
../Ride_Data_Cache/survival_probability_df.msgpack exists
Decay Calculation done in 0.983065128326416
1e
../Ride_Data_Cache/features_orders.msgpack exists
Features created in 0.8833768367767334
1f
../Ride_Data_Cache/radial_spatial_df.msgpack exists
spatial
Spatial Calculation done in 4.708781003952026


### Modelling Code 

We have till now coded the following models 

1. Linear Regression 
2. Linear Regression with L1 Regularization
3. Linear Regression with L2 Regularization
4. Random Forest Regressor 

Now we split the data in a ratio of 75:25

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNet
from sklearn.ensemble import RandomForestRegressor

In [19]:
def build_model(df, target_df, name, seed=0):
    # regression forest
    print(f'building regression model for {name}')
    X = df    
    xtrain, xtest, ytrain, ytest = train_test_split(X, target_df['target'], random_state=seed)

    sc = StandardScaler()

    xtrain_sc = sc.fit_transform(xtrain)

    rr = RandomForestRegressor(random_state=seed)
    rr.fit(xtrain_sc, ytrain)

    xtest_sc = sc.transform(xtest)
    
    print(rr.score(xtest_sc, ytest))
    
    return rr.score(xtest_sc, ytest)

In [20]:
df_final_new_decay_surv.shape, target_df_new_decay_surv.shape
# df_final_no_decay.shape, target_df_no_decay.shape

((1181102, 112), (1181102, 5))

In [25]:
df_list = [df_final_no_decay, df_final_old_decay, df_final_new_decay1, 
           df_final_new_decay10, df_final_new_decay100, df_final_new_decay_surv]
target_df_list = [target_df_no_decay, target_df_old_decay, target_df_new_decay1, 
                  target_df_new_decay10, target_df_new_decay100, target_df_new_decay_surv]
name_list = ['no_decay', 'old_decay', 'new_decay1', 'new_decay10', 'new_decay100', 'survival']

In [26]:
result = {}
for df, target_df, name in zip(df_list, target_df_list, name_list):
    result[name] = build_model(df, target_df, name)

building regression model for no_decay
0.487284067717064
building regression model for old_decay
0.6757007478822953
building regression model for new_decay1
0.6662170568641492
building regression model for new_decay10
0.749684039129092
building regression model for new_decay100
0.7914960448891515
building regression model for survival
0.888966343003949


In [27]:
result

{'no_decay': 0.487284067717064,
 'old_decay': 0.6757007478822953,
 'new_decay1': 0.6662170568641492,
 'new_decay10': 0.749684039129092,
 'new_decay100': 0.7914960448891515,
 'survival': 0.888966343003949}

In [30]:
col = [x.shape[1] for x in df_list]
col

[111, 112, 112, 112, 112]

In [31]:
set(df_list[1].columns).difference(df_list[0].columns)

{'inactive_time'}

In [28]:
df_final_old_decay.head()

,num_total_rides,num_pool_rides,% of pool rides,"(0 days 00:00:44.999999, 0 days 10:36:30]_count","(0 days 10:36:30, 0 days 14:43:06]_count","(0 days 14:43:06, 0 days 18:45:25]_count","(0 days 18:45:25, 0 days 23:59:59]_count","(0 days 00:00:44.999999, 0 days 10:36:30]_sum","(0 days 10:36:30, 0 days 14:43:06]_sum","(0 days 14:43:06, 0 days 18:45:25]_sum",...,"(9509.75, 78107.176](-38.414, -8.521]","(9509.75, 78107.176](-8.521, 14.323]","(9509.75, 78107.176](-99.688, -38.414]","(9509.75, 78107.176](111.762, 137.56]","(9509.75, 78107.176](137.56, 180.0]","(9509.75, 78107.176](14.323, 38.147]","(9509.75, 78107.176](38.147, 60.214]","(9509.75, 78107.176](60.214, 88.383]","(9509.75, 78107.176](88.383, 111.762]",inactive_time
driver_id,,,,,,,,,,,,,,,,,,,,,
0000131d486b69eb77ab6e9e7cca9f4c,3,0.0,0.000000,0,1,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
00002724a19c5f6a54ae8d60a378997e,1,0.0,0.000000,1,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
000042288ba964aa04d14869bd56efb6,5,0.0,0.000000,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.690378
000049d47b79747ad9927ba1bf5f4126,11,1.0,0.090909,0.363636,0.0909091,0.545455,0,0.347145,0.083383,0.569472,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,243.061737
00005adc52a78e1630cfb50e4bd0f89e,4,0.0,0.000000,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [29]:
df_final_new_decay10.head()

,num_total_rides,num_pool_rides,% of pool rides,"(0 days 00:00:44.999999, 0 days 10:36:30]_count","(0 days 10:36:30, 0 days 14:43:06]_count","(0 days 14:43:06, 0 days 18:45:25]_count","(0 days 18:45:25, 0 days 23:59:59]_count","(0 days 00:00:44.999999, 0 days 10:36:30]_sum","(0 days 10:36:30, 0 days 14:43:06]_sum","(0 days 14:43:06, 0 days 18:45:25]_sum",...,"(9509.75, 78107.176](-38.414, -8.521]","(9509.75, 78107.176](-8.521, 14.323]","(9509.75, 78107.176](-99.688, -38.414]","(9509.75, 78107.176](111.762, 137.56]","(9509.75, 78107.176](137.56, 180.0]","(9509.75, 78107.176](14.323, 38.147]","(9509.75, 78107.176](38.147, 60.214]","(9509.75, 78107.176](60.214, 88.383]","(9509.75, 78107.176](88.383, 111.762]",inactive_time
driver_id,,,,,,,,,,,,,,,,,,,,,
0000131d486b69eb77ab6e9e7cca9f4c,3,0.0,0.000000,0,1,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.871019
00002724a19c5f6a54ae8d60a378997e,1,0.0,0.000000,1,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
000042288ba964aa04d14869bd56efb6,5,0.0,0.000000,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75.475753
000049d47b79747ad9927ba1bf5f4126,11,1.0,0.090909,0.363636,0.0909091,0.545455,0,0.347145,0.083383,0.569472,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,333.463457
00005adc52a78e1630cfb50e4bd0f89e,4,0.0,0.000000,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.870189
